# Tokamak PROCESS-Lite 0D — Demo Notebook

This Jupyter notebook demonstrates how to use the **PROCESS-Lite 0D** codebase.

It shows:
1. How to import the subroutines from `src/subroutines/`
2. How to run a parametric scan over (Aspect Ratio, Bt)
3. How to generate high-quality 3D surface plots

This notebook is intended for CDT Fusion Engineering student members and collaborators.

## 1. Setup Path

We add the repository `src/` directory to Python's import path so we can import modules cleanly.

In [ ]:
import sys
import os

# Adjust this path if your notebook is elsewhere
repo_root = os.path.abspath("../")  # one level up from /examples
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("Repo root:", repo_root)
print("Added to Python path:", src_path)

## 2. Import PROCESS-Lite modules

We import the structured subroutines exactly like a Python package.

In [ ]:
from subroutines.geometry_plasma import Shape, Ip_from_q95_PROCESS, greenwald_density, thermal_beta
from subroutines.tf_coils import TFCoils
from subroutines.scanning import scan_A_Bt_grid
from subroutines.plotting import plot_surface

import numpy as np
import matplotlib.pyplot as plt

## 3. Define baseline plasma parameters

These parameters correspond to a realistic **university-scale non-neutronic tokamak**, similar to SMART (University of Seville) and ST-series devices.


In [ ]:
# plasma geometry and physics
a = 0.25     # minor radius [m]
kappa = 1.6  # elongation
delta = 0.0  # triangularity
q95 = 4.0
f_G = 0.4
Te_eV = 100.0
Ti_eV = 50.0

# TF coil configuration
tf = TFCoils(
    N_tf=12,
    N_turn=4,
    I_max_coil=15e3  # Ampere
)

print("TF coil model loaded.")

## 4. Run the (A, Bt) parameter scan

We sweep over:
- Aspect ratio: 1.5 → 3.0
- Toroidal field: 0.25 T → 0.75 T

For each pair, we compute plasma current, density, TF coil current and beta.

In [ ]:
grids = scan_A_Bt_grid(
    A_min=1.5,
    A_max=3.0,
    Bt_min=0.25,
    Bt_max=0.75,
    n_A=60,
    n_Bt=60,
    a=a,
    kappa=kappa,
    delta=delta,
    q95=q95,
    f_G=f_G,
    Te_eV=Te_eV,
    Ti_e_V=Ti_eV,
    tf=tf,
)

print("Scan completed.")

## 5. Extract results for plotting


In [ ]:
A = grids["A"]
Bt = grids["Bt"]
Ip_kA = grids["Ip_MA"] * 1e3
ne = grids["ne"]
Icoil_kA = grids["Icoil"] * 1e-3
beta = grids["beta"]
mask = grids["feasible"]

# mask out infeasible points
Ip_kA = np.where(mask, Ip_kA, np.nan)
ne = np.where(mask, ne, np.nan)
Icoil_kA = np.where(mask, Icoil_kA, np.nan)
beta = np.where(mask, beta, np.nan)

print("Data arrays prepared.")

## 6. Plot surfaces

Each surface shows a fundamental relationship essential for tokamak engineering decisions:
- **Ip(A, Bt)**: How shaping and toroidal field influence plasma current
- **ne(A, Bt)**: Expected operating density
- **TF coil current(A, Bt)**: Engineering coil requirement
- **Beta(A, Bt)**: Rough pressure–magnetic balance indicator


In [ ]:
plot_surface(A, Bt, Ip_kA, "A", "Bt [T]", "Ip [kA]", "Plasma current vs A,Bt")
plot_surface(A, Bt, ne, "A", "Bt [T]", "ne [m^-3]", "Electron density vs A,Bt")
plot_surface(A, Bt, Icoil_kA, "A", "Bt [T]", "I_coil [kA]", "TF coil current vs A,Bt")
plot_surface(A, Bt, beta, "A", "Bt [T]", "beta", "Thermal beta vs A,Bt")
plt.show()

## 7. Next Steps

You can extend this notebook to explore:
- Central solenoid (CS) flux-swing estimates
- PF coil ampere-turns for shaping equilibrium
- Vacuum vessel geometry constraints
- Energy confinement scalings for more realistic temperature predictions

If you'd like, I can generate **another notebook** specifically for:
- TF coil optimization
- CS sizing
- PF equilibrium approximations
- Automatic PDF/PNG export of figures
- A publication-ready multi-panel figure (all 4 surfaces on 1 page).
